# Install and load py4j

In [1]:
#Install and load py4j
#%pip install py4j
from py4j.java_gateway import JavaGateway,GatewayParameters

# Connect java-python (and erode and z3)

Before running this notebook, it is necessary to run erode4Colomoto.jar passing argument the port specified in the next cell (25336)
- E.g.: java -Djava.library.path="native/osx" -jar erode4Colomoto.jar 25336

In [2]:
#Get the current directory, useful later
import os
current_path=os.getcwd()

#Connect python and java
#gateway = JavaGateway()#default port: 25333
jvm_port=25336
gateway = JavaGateway(gateway_parameters=GatewayParameters(port=jvm_port))

erode=gateway.entry_point

#Now, I can call any Java method 'm' from class EntryPointBNColomoto.
# I just need to write: erode.m()

# Load a bnet model in ERODE

In [3]:
#When loading a model, ERODE can create an initial partiton 'USER' such that
# INPUTS:          every input  variable is isolated in a singleton block (plus one block for the other variables)
# OUTPUTS:         every output variable is isolated in a singleton block (plus one block for the other variables)
# INPUTSONEBLOCK:  all input  variables are in the same block (plus one block for the other variables)
# OUTPUTSONEBLOCK: all output variables are in the same block (plus one block for the other variables)
buildUserPartitonOnLoading="INPUTS"
#n=erode.loadBNet("/Users/andrea/Library/CloudStorage/OneDrive-ScuolaSuperioreSant'Anna/DISTR/ERODE-Distr/colomoto/ap-1_else-0_wt.bnet",buildUserPartitonOnLoading)
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet",buildUserPartitonOnLoading)

#After the model has been loaded, we can print it:
modelString=erode.getModelString()
print(modelString)
print()
print()
print()


#We can also access the variables in the model:
speciesNames=erode.getSpeciesNames()
print("The species are:")
for sp in speciesNames:
    print(sp)

ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2



The species are:
Ap
Ap2
Dl
Dl2
Fng
Fng2
N
N2
Ser
Ser2


# Compute a BBE partition

## Partitions

We represent partitions as arrays of integers. 
- Position i represents species i
- We provide a number in each position. 
  - If positions i and j have same number, then the two corresponding species are in the same block

In [4]:
initial_partition = erode.getInitialPartition()
for i in initial_partition:
    print(i)

print()
print("We can ask ERODE to pretty-print the partitions: \n Given that we only have '1', all species belong in the same block")
print(erode.getPartitionString(initial_partition))

print()
print("We now put the first variable in a singleton block")
initial_partition[0]=2
for i in initial_partition:
    print(i)
print(erode.getPartitionString(initial_partition))


print(type(initial_partition))
lst = list(initial_partition)
print(lst)

1
1
1
1
1
1
1
1
1
1

We can ask ERODE to pretty-print the partitions: 
 Given that we only have '1', all species belong in the same block
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



We now put the first variable in a singleton block
2
1
1
1
1
1
1
1
1
1
The partition has 2 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 9
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 


<class 'py4j.java_collections.JavaArray'>
[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Maximal reduction

In [5]:
print("")
print("MAXIMAL REDUCTION")
print("")

initial_partition = erode.getInitialPartition()
print("We start from partition:")
print(erode.getPartitionString(initial_partition))

print("")
print("")

#partition=erode.computeBBEPartition("MAX")
partition=erode.computeBBEPartition(initial_partition)
print("The obtained partition is:")
print(erode.getPartitionString(partition))


MAXIMAL REDUCTION

We start from partition:
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 




The obtained partition is:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 




## User-def partition generated when loading the model

In [6]:
print("")
print("USE PARTITION CREATED WHEN LOADING OF THE MODEL")
print("")
initial_partition = erode.getUserPartition()
print("We start from partition:")
print(erode.getPartitionString(initial_partition))

print("")
print("")

partition=erode.computeBBEPartition("USER")
#partition=erode.computeBBEPartition(initial_partition)
print("The obtained partition is:")
print(erode.getPartitionString(partition))


USE PARTITION CREATED WHEN LOADING OF THE MODEL

We start from partition:
The partition has 3 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 




The obtained partition is:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 




## User-def partition generated AFTER loading the model

### Inputs-preserving

In [7]:
print("")
print("USE PARTITION THAT PRESERVES INPUTS")
print("")
initial_partition = erode.buildPartition("INPUTS")
print("We start from partition:")
print(erode.getPartitionString(initial_partition))

print("")
print("")

partition=erode.computeBBEPartition(initial_partition)
print("The obtained partition is:")
print(erode.getPartitionString(partition))


USE PARTITION THAT PRESERVES INPUTS

We start from partition:
The partition has 3 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 




The obtained partition is:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 




### Separating inputs and non-inputs species

In [8]:
print("")
print("USE PARTITION THAT SEPARATES INPUTS/non-INPUTS")
print("")
initial_partition = erode.buildPartition("INPUTSONEBLOCK")
print("We start from partition:")
print(erode.getPartitionString(initial_partition))

print("")
print("")

partition=erode.computeBBEPartition(initial_partition)
print("The obtained partition is:")
print(erode.getPartitionString(partition))


USE PARTITION THAT SEPARATES INPUTS/non-INPUTS

We start from partition:
The partition has 2 blocks out of 10 species:
Block 1, Size: 2
0-Ap 
1-Ap2 

Block 2, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 




The obtained partition is:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 




### Outputs-preserving

Same as `INPUTS`, but shall use `OUTPUTS`

### Separating inputs and non-inputs species

Same as `INPUTSONEBLOCK`, but shall use `OUTPUTSONEBLOCK`

## Programmatically generated initial partitions

In [9]:
print("")
print("USE PARTITION CREATED WHEN LOADING OF THE MODEL")
print("")
initial_partition = erode.getInitialPartition()
initial_partition[4]=2
print("We start from partition:")
print(erode.getPartitionString(initial_partition))

print("")
print("")

partition=erode.computeBBEPartition(initial_partition)
print("The obtained partition is:")
print(erode.getPartitionString(partition))


USE PARTITION CREATED WHEN LOADING OF THE MODEL

We start from partition:
The partition has 2 blocks out of 10 species:
Block 1, Size: 9
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 

Block 2, Size: 1
4-Fng 




The obtained partition is:
The partition has 9 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 1
4-Fng 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 

Block 9, Size: 1
8-Ser 




# Compute reduced model

## Maximal

In [10]:
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet")

originalModelString=erode.getModelString()
print("\n##Original model:")
print(originalModelString)

obtained_partition=erode.computeBBEPartition("MAX")
print("\n##Computed BBE:")
print(erode.getPartitionString(obtained_partition))

erode.computeBBEReducedModel(obtained_partition)
reducedModelString=erode.getModelString()
print("\n##Reduced model:")
print(reducedModelString)


erode.writeBNet(current_path+"/ap-1_else-0_wt_BBE.bnet")


##Original model:
ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2

##Computed BBE:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 



##Reduced model:
ap-1_else-0_wt.bnetBBE: 8 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Fng2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Fng))


## Inputs-preserving

In [11]:
guessPrep="INPUTS"
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet",guessPrep)

originalModelString=erode.getModelString()
print("\n##Original model:")
print(originalModelString)

initial_partition= erode.getUserPartition()
print("\n##Initial partition computed at loading time for",guessPrep)
print(erode.getPartitionString(initial_partition))

initial_partition= erode.buildPartition(guessPrep)
print("\n##Initial partition computed later for",guessPrep)
print(erode.getPartitionString(initial_partition))



obtained_partition=erode.computeBBEPartition(initial_partition)
print("\n##Computed BBE:")
print(erode.getPartitionString(obtained_partition))

erode.computeBBEReducedModel(obtained_partition)
reducedModelString=erode.getModelString()
print("\n##Reduced model:")
print(reducedModelString)


# erode.writeBNet(current_path+"/ap-1_else-0_wt_BBE.bnet")


##Original model:
ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2

##Initial partition computed at loading time for INPUTS
The partition has 3 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Initial partition computed later for INPUTS
The partition has 3 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Computed BBE:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 



##Reduced model:
ap-1_el

## Inputs one-block (separating inputs and non-inputs)

In [12]:
guessPrep="INPUTSONEBLOCK"
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet",guessPrep)

originalModelString=erode.getModelString()
print("\n##Original model:")
print(originalModelString)

initial_partition= erode.getUserPartition()
print("\n##Initial partition computed at loading time for",guessPrep)
print(erode.getPartitionString(initial_partition))

initial_partition= erode.buildPartition(guessPrep)
print("\n##Initial partition computed later for",guessPrep)
print(erode.getPartitionString(initial_partition))



obtained_partition=erode.computeBBEPartition(initial_partition)
print("\n##Computed BBE:")
print(erode.getPartitionString(obtained_partition))

erode.computeBBEReducedModel(obtained_partition)
reducedModelString=erode.getModelString()
print("\n##Reduced model:")
print(reducedModelString)


# erode.writeBNet(current_path+"/ap-1_else-0_wt_BBE.bnet")


##Original model:
ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2

##Initial partition computed at loading time for INPUTSONEBLOCK
The partition has 2 blocks out of 10 species:
Block 1, Size: 2
0-Ap 
1-Ap2 

Block 2, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Initial partition computed later for INPUTSONEBLOCK
The partition has 2 blocks out of 10 species:
Block 1, Size: 2
0-Ap 
1-Ap2 

Block 2, Size: 8
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Computed BBE:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 



##Reduced model:
ap-1_else-0_wt.bnetBBE: 8 s

## Outputs-preserving

In [13]:
guessPrep="OUTPUTS"
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet",guessPrep)

originalModelString=erode.getModelString()
print("\n##Original model:")
print(originalModelString)

initial_partition= erode.getUserPartition()
print("\n##Initial partition computed at loading time for",guessPrep)
print(erode.getPartitionString(initial_partition))

initial_partition= erode.buildPartition(guessPrep)
print("\n##Initial partition computed later for",guessPrep)
print(erode.getPartitionString(initial_partition))



obtained_partition=erode.computeBBEPartition(initial_partition)
print("\n##Computed BBE:")
print(erode.getPartitionString(obtained_partition))

erode.computeBBEReducedModel(obtained_partition)
reducedModelString=erode.getModelString()
print("\n##Reduced model:")
print(reducedModelString)


# erode.writeBNet(current_path+"/ap-1_else-0_wt_BBE.bnet")


##Original model:
ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2

##Initial partition computed at loading time for OUTPUTS
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Initial partition computed later for OUTPUTS
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Computed BBE:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 



##Reduced model:
ap-1_else-0_wt.bnetBBE: 8 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, 

## Outputs one-block (separating outputs and non-outputs)

In [14]:
guessPrep="OUTPUTSONEBLOCK"
erode.loadBNet(current_path+"/ap-1_else-0_wt.bnet",guessPrep)

originalModelString=erode.getModelString()
print("\n##Original model:")
print(originalModelString)

initial_partition= erode.getUserPartition()
print("\n##Initial partition computed at loading time for",guessPrep)
print(erode.getPartitionString(initial_partition))

initial_partition= erode.buildPartition(guessPrep)
print("\n##Initial partition computed later for",guessPrep)
print(erode.getPartitionString(initial_partition))



obtained_partition=erode.computeBBEPartition(initial_partition)
print("\n##Computed BBE:")
print(erode.getPartitionString(obtained_partition))

erode.computeBBEReducedModel(obtained_partition)
reducedModelString=erode.getModelString()
print("\n##Reduced model:")
print(reducedModelString)


# erode.writeBNet(current_path+"/ap-1_else-0_wt_BBE.bnet")


##Original model:
ap-1_else-0_wt.bnet: 10 species.
Species:
[Ap, Ap2, Dl, Dl2, Fng, Fng2, N, N2, Ser, Ser2]
	Ap:	true
	Ap2:	false
	Dl:	(!Ap&N)
	Dl2:	(!Ap2&N2)
	Fng:	Ap
	Fng2:	Ap2
	N:	((!Fng&Ser2) | (Fng&Dl2))
	N2:	((!Dl&!Fng2) | (Dl&Ser))
	Ser:	Ap
	Ser2:	Ap2

##Initial partition computed at loading time for OUTPUTSONEBLOCK
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Initial partition computed later for OUTPUTSONEBLOCK
The partition has 1 block out of 10 species:
Block 1, Size: 10
0-Ap 
1-Ap2 
2-Dl 
3-Dl2 
4-Fng 
5-Fng2 
6-N 
7-N2 
8-Ser 
9-Ser2 



##Computed BBE:
The partition has 8 blocks out of 10 species:
Block 1, Size: 1
0-Ap 

Block 2, Size: 1
1-Ap2 

Block 3, Size: 1
2-Dl 

Block 4, Size: 1
3-Dl2 

Block 5, Size: 2
4-Fng 
8-Ser 

Block 6, Size: 2
5-Fng2 
9-Ser2 

Block 7, Size: 1
6-N 

Block 8, Size: 1
7-N2 



##Reduced model:
ap-1_else-0_wt.bnetBBE: 8 species.
Species:
[Ap, Ap2, Dl, Dl2

# Two concurrent ERODE 

To run the next cell, you shall run erode4Colomoto.jar passing argument 25338
- E.g.: java -Djava.library.path="native/osx" -jar erode4Colomoto.jar 25338